In [19]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, log_loss

In [20]:
def testmymodel(model, features, labels):
    try:
        X = np.load(features)
    except FileNotFoundError:
        raise("didnt find the features file")

    try:
        Y = np.load(labels)
    except FileNotFoundError:
        raise("didnt find the labels file")

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
    model.fit(X=X_train, y=y_train)
    prediction_accuracy = model.score(X_test, y_test)
    return prediction_accuracy

In [21]:
try:
    X = np.load("cifar10_features.npy")
except FileNotFoundError:
    raise("didnt find the features file")

try:
    Y = np.load("cifar10_labels.npy")
except FileNotFoundError:
    raise("didnt find the labels file")

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [22]:
softMax = LogisticRegression(multi_class='multinomial', max_iter=1000)
prediction_accuracy = testmymodel(softMax, "cifar10_features.npy", "cifar10_labels.npy")
print(f"the accuracy of the softmax = {prediction_accuracy}")

the accuracy of the softmax = 0.9629333333333333


In [23]:
%%timeit
softMax.fit(X=X_train, y=y_train)

1.81 s ± 147 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
ova = LogisticRegression(multi_class='ovr', max_iter=1000)
prediction_accuracy2 = testmymodel(ova, "cifar10_features.npy", "cifar10_labels.npy")
print(f"the accuracy of the one vs all = {prediction_accuracy2}")

the accuracy of the one vs all = 0.961


In [25]:
%%timeit
ova.fit(X=X_train, y=y_train)

877 ms ± 76.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


as we see above the two algorithms have the same accuracy but there is a huge gap in the run time (the one vs all  the run time of the softmax)

In [26]:
softMax_loss = log_loss(y_true=y_test, y_pred=softMax.predict_proba(X_test))
print(f"soft max loss = {softMax_loss}")
ova_loss = log_loss(y_true=y_test, y_pred=ova.predict_proba(X_test))
print(f"one versus all loss = {ova_loss}")

soft max loss = 0.10768597462451053
one versus all loss = 0.13459723836922313


In [27]:
softMax_f1_mean = f1_score(y_true=y_test, y_pred=softMax.predict(X_test), average='macro')
print(f"soft max f1-mean = {softMax_f1_mean}")
ova_f1_mean = f1_score(y_true=y_test, y_pred=ova.predict(X_test), average='macro')
print(f"one versus all f1-mean = {ova_f1_mean}")

soft max f1-mean = 0.9631573951463324
one versus all f1-mean = 0.961210892245097


In [28]:
ova_confusion_matrix = confusion_matrix(y_true=y_test, y_pred=ova.predict(X_test))
print(ova_confusion_matrix)

[[1402    2   13    5    5    3    2   10   17    5]
 [   2 1464    4    1    0    1    0    1    5    7]
 [  11    1 1370   15   16   15    5    3    1    3]
 [  14    5   15 1450   12   51    5   10    3    4]
 [   6    0   13   15 1466    5    4    9    0    1]
 [   2    3   13   45   12 1437    8   11    2    1]
 [   5    5    9   17    4    6 1415    0    1    1]
 [   3    0    4   17    9    7    0 1456    0    1]
 [  13    2    1    5    1    0    2    0 1480    6]
 [  12    7    5    6    0    4    2    3    5 1475]]


the two classes that the model finds it the most hard to tell the differences between them are the classes with the values 3 and 5 (cats and dogs), it predicts 51 cats as dogs and 45 dogs as cats

In [30]:
Xnew = []
Ynew = []
for i in range(len(Y)):
    if Y[i] == 3 or Y[i] == 5 :
        Xnew.append(X[i])
        Ynew.append(Y[i])

Xnew_train, Xnew_test, ynew_train, ynew_test = train_test_split(Xnew, Ynew, test_size=0.3, random_state=42)
ovaNew = LogisticRegression(multi_class='ovr', max_iter=1000)
ovaNew.fit(X=Xnew_train, y=ynew_train)

prediction_accuracy_new = ovaNew.score(Xnew_test, ynew_test)
print(f"one versus all new {prediction_accuracy_new}")

ova_confusion_matrix_new = confusion_matrix(y_true=ynew_test, y_pred=ovaNew.predict(Xnew_test))
print(ova_confusion_matrix_new)

one versus all new 0.9633333333333334
[[1446   57]
 [  53 1444]]


the accuracy is the same and the confusion matrix is the same too (we think the problem is not because of the model its because of the features we have here)